In [1]:
#Import libraries 
import pandas as pd 
import numpy as np
import os 

# Following with reading 

In [2]:
#Path to Instacart project 
path = r'/Users/libertyonia/Documents/Instacart Basket Analysis'

In [3]:
#Import ords_prods_merged data frame, but only the first 1,000,000 rows 
df_ords_prods_merge = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data',' orders_products_merged.pkl'))
df = df_ords_prods_merge[:1000000]

In [4]:
#First 5 rows of ords_prods_merged data frame
df.head(5)

,order_id,user_id,eval_set,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both
1,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both
2,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both
3,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both
4,3367565,1,prior,6,2,7,19.0,196,1,1,Soda,77,7,9.0,both


In [5]:
#Create new price_label() function 
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [6]:
#Apply price_label() function on the df subsetto create new column price_range
df['price_range'] = df.apply(price_label, axis=1)

/var/folders/7x/7rtx5kls68d6l5dc55xbmjph0000gn/T/ipykernel_50021/2059542084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [7]:
#Check price range values by creating a frequency 
df['price_range'].value_counts()

Mid-range product    631969
Low-range product    368031
Name: price_range, dtype: int64

In [8]:
#Find the most expensive subset 
df['prices'].max()

14.0

In [9]:
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

/var/folders/7x/7rtx5kls68d6l5dc55xbmjph0000gn/T/ipykernel_50021/1169838859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [10]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [11]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [12]:
df['price_range_loc'].value_counts(dropna = False)

Mid-range product    631969
Low-range product    368031
Name: price_range_loc, dtype: int64

In [13]:
#Apply loc() function to all rows in the df_ords_prods_merge dataframe 
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'
df_ords_prods_merge.loc[(df_ords_prods_merge['prices'] <= 15) & (df_ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [14]:
df_ords_prods_merge['price_range_loc'].value_counts(dropna = False)

Mid-range product     20462144
Low-range product      9476774
High-range product      389845
Name: price_range_loc, dtype: int64

In [15]:
df_ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc
0,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product
1,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product
2,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product
3,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product
4,3367565,1,prior,6,2,7,19.0,196,1,1,Soda,77,7,9.0,both,Mid-range product


## If-Statements with For-Loops 

In [16]:
#Frequency of orders per week 
#0 is Saturday, therefore, Saturday is the busiest day of the week 
df_ords_prods_merge['order_day_of_week'].value_counts(dropna = False)

0    5779087
1    5303718
6    4190948
5    3952326
2    3947564
3    3600589
4    3554531
Name: order_day_of_week, dtype: int64

In [17]:
#This code will generate a list of Busiest, least busy, and regularly busy in the df_ords_prods_merge dataframe  days 

result = []

for value in df_ords_prods_merge["order_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [19]:
#Merge result list into the ords_prods_merge dataframe as a new column called busiest_day
df_ords_prods_merge['busiest_day'] = result 

In [20]:
#Check the frequency of the busiest_day column 
df_ords_prods_merge['busiest_day'].value_counts(dropna = False)

Regularly busy    20995145
Busiest day        5779087
Least busy         3554531
Name: busiest_day, dtype: int64

In [21]:
#Check if columns were added to the df_ords_prods_merge dataframe 
df_ords_prods_merge.head(3)

,order_id,user_id,eval_set,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day
0,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy
1,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy
2,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy


# Task 4.7

### Question 2
Now, they want “Busiest day” to become “Busiest days” (plural). This label should correspond with the two busiest days of the week as opposed to the single busiest day. At the same time, they’d also like to know the two slowest days. Create a new column for this using a suitable method.

In [22]:
#Generate a list with the two busiest days or two slowest days
result_2 = [] 

for value in df_ords_prods_merge["order_day_of_week"]:
    if value == 0:
        result_2.append("Busiest days")
    elif value == 1:
        result_2.append("Busiest days")
    elif value == 4:
        result_2.append("Slowest days")
    elif value == 3:
        result_2.append("Slowest days")
    else: 
        result_2.append("Regularly busy")

In [23]:
#Merge result_2 list with the df_ords_prods_merge dataframe 
df_ords_prods_merge['busiest_days'] = result_2

### Question 3
Check the values of this new column for accuracy. Note any observations in markdown format.

In [24]:
#Counts of days 
df_ords_prods_merge['busiest_days'].value_counts(dropna = False)

Regularly busy    12090838
Busiest days      11082805
Slowest days       7155120
Name: busiest_days, dtype: int64

### Answer 3 
The answer to the above cell is consistent to the results of "" df_ords_prods_merge['order_day_of_week'].value_counts(dropna = False) "". The two busiest days (0 & 1) have a total of  5779087 + 5303718 = 11082805 instances. The two slowest days (3 & 4) have a total of 3600589 + 3554531 = 7155120 instances. 

### Question 4 
When too many users make Instacart orders at the same time, the app freezes. The senior technical officer at Instacart wants you to identify the busiest hours of the day. Rather than by hour, they want periods of time labeled “Most orders,” “Average orders,” and “Fewest orders.” Create a new column containing these labels called “busiest_period_of_day.”

In [25]:
#Check which hours have the most orders
df_ords_prods_merge['order_hour_of_day'].value_counts(dropna = False)

10    2593725
11    2564597
14    2517238
15    2487586
13    2487500
12    2445841
16    2364969
9     2311334
17    1943858
8     1622394
18    1520954
19    1169224
20     910005
7      844665
21     746254
22     592432
23     375889
6      274801
0      203460
1      108110
5       82706
2       63961
4       49400
3       47860
Name: order_hour_of_day, dtype: int64

In [26]:
#Top 3 busiest hours are labeled "busiest hours"
#Bottom 3 slowest hours are labeled "slowest hours"
#All other hours are "average hours"
result_3 = []

for value in df_ords_prods_merge['order_hour_of_day']:
    if value == 10 or value == 11 or value == 14:
        result_3.append('Busiest hours')
    elif value == 2 or value == 4 or value == 3: 
        result_3.append('Slowest hours')
    else:
        result_3.append('Average hours')

In [27]:
#Merge result_3 to df_ords_prods_merge dataframe 
df_ords_prods_merge['busiest_period_of_day'] = result_3

### Question 5 
Print the frequency for this new column 

In [28]:
#frequency of busiest_period_of_day
df_ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

Average hours    22491982
Busiest hours     7675560
Slowest hours      161221
Name: busiest_period_of_day, dtype: int64

In [29]:
df_ords_prods_merge.head(3)

,order_id,user_id,eval_set,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average hours
1,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average hours
2,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average hours


In [30]:
#Export the dataframe with new additional columns 
df_ords_prods_merge.to_pickle(os.path.join(path,'02 Data','Prepared Data','ords_prods_merge_4.7'))